# Import Required Libraries
Import the necessary libraries, including requests, pandas, numpy, scipy, and sklearn.

In [6]:
# Import Required Libraries
import requests
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

# Authenticate and Retrieve Portfolio
Authenticate with the Robinhood API and retrieve the current portfolio tickers.

In [3]:
# Authenticate and Retrieve Portfolio

import robin_stocks.robinhood as r

# Define a function to authenticate with Robinhood API
def authenticate_robinhood(username, password):
    login = r.login(username, password)
    return login

# Define a function to retrieve the current portfolio tickers
def get_portfolio_tickers():
    positions_data = r.account.build_holdings()
    tickers = list(positions_data.keys())
    return tickers

# Example usage
username = input("Enter your Robinhood username: ")
password = input("Enter your Robinhood password: ")
authenticate_robinhood(username, password)
tickers = get_portfolio_tickers()
print(tickers)


['META', 'AAPL', 'SPOT', 'NVDA', 'SHOP', 'GOOGL', 'BRK.B', 'RBLX', 'GS', 'Z', 'LLY', 'AVAV', 'ARM', 'SMR', 'VRSK', 'JPM', 'TGS', 'MU', 'CEG']


# Fetch Historical Data
Fetch historical data for each ticker in the portfolio using an appropriate API.

In [64]:
import os
import pickle
from datetime import datetime, timedelta

# Fetch Historical Data

# Define a function to fetch historical data for a given ticker
def fetch_historical_data(ticker, interval='day', span='5year'):
    historical_data = r.stocks.get_stock_historicals(ticker, interval=interval, span=span)
    historical_data = pd.DataFrame(historical_data)
    
    return historical_data

# Define a function to load data from cache if it exists and is younger than 1 day
def load_from_cache(ticker):
    cache_file = f"{ticker}_historical_data.pkl"
    if os.path.exists(cache_file):
        file_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - file_time < timedelta(days=1):
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
    return None

# Define a function to save data to cache
def save_to_cache(ticker, data):
    cache_file = f"{ticker}_historical_data.pkl"
    with open(cache_file, 'wb') as f:
        pickle.dump(data, f)




# Calculate Metrics
Calculate the slope, variance, standard deviation, and maximum drawdown for each ticker.

In [ ]:
from datetime import timezone


def linear_func(x, m, c):
    return m * x + c

def exponential_func(x, a, b):
    return a * np.exp(b * x)


# Define a function to calculate metrics for multiple time ranges
def calculate_metrics_for_multiple_ranges(historical_data):
    time_ranges = {
        '5_years': datetime.now() - timedelta(days=5*365),
        '3_years': datetime.now() - timedelta(days=3*365),
        '1_year': datetime.now() - timedelta(days=365),
        '6_months': datetime.now() - timedelta(days=6*30),
        '3_months': datetime.now() - timedelta(days=3*30)
    }
    
    aggregated_results = pd.DataFrame()
    
    for range_name, start_date in time_ranges.items():
        metrics_df = calculate_regression_metrics_for_range(historical_data, start_date)
        metrics_df['time_range'] = range_name
        aggregated_results = pd.concat([aggregated_results, metrics_df], ignore_index=True)
    
    return aggregated_results

def linear_func(x, m, c):
    return m * x + c

def exponential_func(x, a, b):
    return a * np.exp(b * x)

#Define a function to calculate metrics for each ticker for a given time range
def calculate_regression_metrics_for_range(historical_data, start_date):
    results = []
    for ticker, data in historical_data.items():
        
        # Convert start_date to the same timezone as data.index
        start_date = pd.to_datetime(start_date).tz_convert(data.index.tz)
        
        # Filter data for the given time range
        data_range = data[data.index >= start_date]
        
        if data_range.empty:
            continue
        
        # Convert date to ordinal for fitting
        data_range['date'] = data_range.index.to_series().apply(lambda date: date.toordinal())
        x_data = data_range['date']
        y_data = data_range['close_price'].astype(float)

        # Fit the linear curve
        popt_linear = np.polyfit(x_data, y_data, 1)
        
        # Fit the exponential curve
        popt_exp, _ = curve_fit(exponential_func, x_data, y_data, maxfev=10000)
        
        # Calculate metrics for linear fit
        y_pred_linear = linear_func(x_data, *popt_linear)
        slope = popt_linear[0]
        variance = np.var(y_pred_linear)
        std_dev = np.std(y_pred_linear)
        max_drawdown = np.min(y_pred_linear) - np.max(y_pred_linear)
        
        # Calculate metrics for exponential fit
        exp_param = popt_exp[1]  # Exponential growth rate
        
        # Append results
        results.append({
            'ticker': ticker,
            'slope': slope,
            'exp_param': exp_param,
            'variance': variance,
            'standard_deviation': std_dev,
            'maximum_drawdown': max_drawdown
        })
    

    
    return pd.DataFrame(results)

# Define a function to calculate metrics for multiple time ranges
def calculate_metrics_for_multiple_ranges(historical_data):
    time_ranges = {
        '5_years': datetime.now(tz=timezone.utc) - timedelta(days=5*365),
        '3_years': datetime.now(tz=timezone.utc) - timedelta(days=3*365),
        '1_year': datetime.now(tz=timezone.utc) - timedelta(days=365),
        '6_months': datetime.now(tz=timezone.utc) - timedelta(days=6*30),
        '3_months': datetime.now(tz=timezone.utc) - timedelta(days=3*30)
    }
    aggregated_results = pd.DataFrame()
    
    for range_name, start_date in time_ranges.items():
        metrics_df = calculate_regression_metrics_for_range(historical_data, start_date)
        metrics_df['time_range'] = range_name
        aggregated_results = pd.concat([aggregated_results, metrics_df], ignore_index=True)
    
    return aggregated_results





/tmp/ipykernel_1168345/2133515577.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda/envs/azureml_py38/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

overflow encountered in exp

/anaconda/envs/azureml_py38/lib/python3.9/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:

Covariance of the parameters could not be estimated



In [58]:
# Example usage
historical_data = {}
for ticker in tickers:
    data = load_from_cache(ticker)
    data = None
    if data is None:
        data = fetch_historical_data(ticker)
        save_to_cache(ticker, data)
    historical_data[ticker] = data
    print(f"Fetched data for {ticker}")

# Display the first few rows of historical data for the first ticker
first_ticker = tickers[0]
historical_data[first_ticker].head()

# Calculate metrics for multiple time ranges
aggregated_results = calculate_metrics_for_multiple_ranges(historical_data)

pivoted_results = aggregated_results.pivot(index='ticker', columns='time_range', values=['slope', 'exp_param', 'variance', 'standard_deviation', 'maximum_drawdown'])
pivoted_results

slope                                         exp_param  \
time_range    1_year  3_months   3_years   5_years  6_months    1_year   
ticker                                                                   
AAPL        0.223789  0.154475  0.079731  0.071666  0.145273       1.0   
ARM         0.086434  0.133196  0.106394  0.045635  0.109620       1.0   
AVAV        0.136736 -0.718977  0.121194  0.057946 -0.123619       1.0   
BRK.B       0.223771 -0.132681  0.168472  0.140429  0.091331       1.0   
CEG         0.332586  0.781764  0.193706  0.102284  0.530993       1.0   
GOOGL       0.108011  0.352636  0.071813  0.049584  0.234325       1.0   
GS          0.641326  0.389513  0.211872  0.144978  0.852285       1.0   
JPM         0.195807  0.211988  0.111691  0.054717  0.299068       1.0   
LLY         0.222644 -0.426864  0.674020  0.433744 -1.082885       1.0   
META        0.445278  0.680924  0.462670  0.147592  0.668180       1.0   
MU         -0.024019 -0.063542  0.050940  0.024640  0.021502       1.0   
NVDA        0.204188 -0.048926  0.126729  0.061946  0.188153       1.0   
RBLX        0.062325  0.188655  0.005449 -0.022265  0.152707       1.0   
SHOP        0.100139  0.168251  0.047297 -0.020403  0.309997       1.0   
SMR         0.061466 -0.023929  0.002248  0.005007  0.112061       1.0   
SPOT        0.688579  0.774393  0.312019  0.042941  1.103576       1.0   
TGS         0.045203  0.074888  0.018187  0.010275  0.087803       1.0   
VRSK        0.150589 -0.120509  0.103681  0.056436  0.103905       1.0   
Z           0.090924  0.061837  0.023842 -0.025944  0.185669       1.0   

                                              ... standard_deviation  \
time_range 3_months 3_years 5_years 6_months  ...             1_year   
ticker                                        ...                      
AAPL            1.0     1.0     1.0      1.0  ...          23.216240   
ARM             1.0     1.0     1.0      1.0  ...           8.966773   
AVAV            1.0     1.0     1.0      1.0  ...          14.185198   
BRK.B           1.0     1.0     1.0      1.0  ...          23.214388   
CEG             1.0     1.0     1.0      1.0  ...          34.503069   
GOOGL           1.0     1.0     1.0      1.0  ...          11.205239   
GS              1.0     1.0     1.0      1.0  ...          66.532243   
JPM             1.0     1.0     1.0      1.0  ...          20.313332   
LLY             1.0     1.0     1.0      1.0  ...          23.097436   
META            1.0     1.0     1.0      1.0  ...          46.193900   
MU              1.0     1.0     1.0      1.0  ...           2.491802   
NVDA            1.0     1.0     1.0      1.0  ...          21.182792   
RBLX            1.0     1.0     1.0      1.0  ...           6.465659   
SHOP            1.0     1.0     1.0      1.0  ...          10.388601   
SMR             1.0     1.0     1.0      1.0  ...           6.376636   
SPOT            1.0     1.0     1.0      1.0  ...          71.434325   
TGS             1.0     1.0     1.0      1.0  ...           4.689476   
VRSK            1.0     1.0     1.0      1.0  ...          15.622328   
Z               1.0     1.0     1.0      1.0  ...           9.432584   

                                                         maximum_drawdown  \
time_range   3_months     3_years     5_years   6_months           1_year   
ticker                                                                      
AAPL         3.876692   25.088725   37.640972   7.295974       -80.564037   
ARM          3.342676   33.478634   23.968913   5.505398       -31.116126   
AVAV        18.043342   38.135596   30.434846   6.208485       -49.224886   
BRK.B        3.329729   53.012278   73.756692   4.586868       -80.557609   
CEG         19.619032   60.952687   53.722103  26.667843      -119.731122   
GOOGL        8.849700   22.597135   26.042767  11.768402       -38.883959   
GS           9.775155   66.668776   76.146131  42.803983      -230.877439   
JPM          5.320010   35.1451

In [57]:
aggregated_results.sort_values('slope', ascending=False)


,ticker,slope,exp_param,variance,standard_deviation,maximum_drawdown,time_range
59,SPOT,1.103576,1.0,3071.875645,55.424504,-193.125865,6_months
65,GS,0.852285,1.0,1832.180962,42.803983,-149.149846,6_months
94,CEG,0.781764,1.0,384.906417,19.619032,-66.449967,3_months
78,SPOT,0.774393,1.0,377.681572,19.434031,-65.823367,3_months
40,SPOT,0.688579,1.0,5102.862761,71.434325,-247.888441,1_year
...,...,...,...,...,...,...,...
68,AVAV,-0.123619,1.0,38.545287,6.208485,-21.633374,6_months
82,BRK.B,-0.132681,1.0,11.087094,3.329729,-11.277843,3_months
86,LLY,-0.426864,1.0,114.757905,10.712512,-36.283444,3_months
87,AVAV,-0.718977,1.0,325.562174,18.043342,-61.113079,3_months


In [63]:
import plotly.express as px

# Sort the time ranges
time_ranges_order = ['3_months', '6_months', '1_year', '3_years', '5_years']
aggregated_results['time_range'] = pd.Categorical(aggregated_results['time_range'], categories=time_ranges_order, ordered=True)
sorted_results = aggregated_results.sort_values('time_range', ascending=False)

# Create the plot
fig = px.line(sorted_results, x='time_range', y='slope', color='ticker', markers=True)

fig.update_layout(
    title="Slope of Different Tickers Over Various Time Ranges",
    xaxis_title="Time Range",
    yaxis_title="Slope"
)

fig.show()
